In [1]:
import itertools
import openturns as ot
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from dependence.tests import func_sum
from dependence.tests.test_functions import func_cum_sum_weight, func_spec
from dependence import ConservativeEstimate
from dependence import quantile_func
from dependence import iterative_vine_minimize
from dependence.dependence_plot import matrix_plot_input, matrix_plot_quantities
from dependence.dependence_plot import plot_iterative_results
from dependence.dependence_plot import set_style_paper

%matplotlib inline
%load_ext autoreload
%autoreload 2

random_state = 12
np.random.seed(random_state)

In [2]:
dim = 20
margins = []
for i in range(dim):
    margins.append(ot.Uniform(0, 1.))
    
margins[0] = ot.Uniform(0, 1.)
alpha = 0.1
q_func = quantile_func(alpha)
families = np.tril(np.ones((dim, dim)), k=1)

In [3]:
n_influencial_pairs = 2
val_influencial = 1.
val_non_influencial = 0.0
corr_dim = int(dim * (dim-1)/2)

all_pairs = np.asarray(list(itertools.combinations(range(dim), 2)))
selected_pairs = all_pairs[np.random.choice(range(corr_dim), size=n_influencial_pairs, replace=False)]
selected_pairs = np.asarray([[0, dim-2], [1, dim-2]])
# selected_pairs = all_pairs

power = np.zeros((dim, dim))
for i in range(1, dim):
    for j in range(i):
        if [j, i] in selected_pairs.tolist():
            print([i, j])
            power[i, j] = val_influencial
        else:
            power[i, j] = val_non_influencial
test_func = lambda x: func_cum_sum_weight(x, power)
print(power)

[18, 0]
[18, 1]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [ ]:
n = 30000
quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)
indep_result = quant_estimate.independence(n_input_sample=n, q_func=q_func, random_state=random_state)
print(indep_result.quantity)

2.2863008627512067


In [ ]:
algorithm_parameters = {
    "n_input_sample": 5000,
    "n_dep_param_init": 10,
    "max_n_pairs": 5,
    "grid_type": 'vertices',
    "q_func": q_func,
    "delta": 0.1,
    "n_add_pairs": 2,
    "n_remove_pairs": 3,
    "adapt_vine_structure": True,
    "with_bootstrap": False,
    "verbose": True,
    "iterative_save": False,
    "iterative_load": False,
    "load_input_samples": False,
    "keep_input_samples": False
}

quant_estimate = ConservativeEstimate(model_func=test_func, margins=margins, families=families)

iterative_results = iterative_vine_minimize(estimate_object=quant_estimate, **algorithm_parameters)

In [ ]:
K = 200
n = 20000

In [ ]:
grid_type = 'lhs'
grid_result_lhs = quant_estimate.gridsearch(K, n, q_func=q_func, grid_type='lhs', keep_input_samples=False)
print('Min quantile: {0}\nMin Kendall: {1}'.format(grid_result_lhs.min_result.quantity, grid_result_lhs.min_result.kendall_tau))

In [ ]:
grid_result_vertices = quant_estimate.gridsearch(K, n, q_func=q_func, grid_type='vertices', keep_input_samples=False)
print('Min quantile: {0}\nMin Kendall: {1}'.format(grid_result_vertices.min_result.quantity, grid_result_vertices.min_result.kendall_tau))

In [ ]:
kendalls_lhs = grid_result_lhs.kendalls
kendalls_vertices = grid_result_vertices.kendalls
dev_kendall_lhs = abs(kendalls_lhs).mean(axis=1)
dev_kendall_vertices = abs(kendalls_vertices).mean(axis=1)
quantities_lhs = grid_result_lhs.quantities
quantities_vertices = grid_result_vertices.quantities
quantity_indep = indep_result.quantity

In [ ]:
min_kendall_lhs = grid_result_lhs.min_result.kendall_tau
min_kendall_vertices = grid_result_vertices.min_result.kendall_tau
min_dev_kendall_lhs = np.abs(min_kendall_lhs).mean()
min_dev_kendall_vertices = np.abs(min_kendall_vertices).mean()
min_quantity_lhs = grid_result_lhs.min_result.quantity
min_quantity_vertices = grid_result_vertices.min_result.quantity

In [ ]:
set_style_paper()
fig, ax = plt.subplots(figsize=(7, 4))    
ax.plot(dev_kendall_lhs, quantities_lhs, 'g.', label='EGS K=%d' % (K))
ax.plot(min_dev_kendall_lhs, min_quantity_lhs, 'go', label='Min EGS')
ax.plot(dev_kendall_vertices, quantities_vertices, 'r.', label='BEGS K=%d' % (min(K, 3**dim-1)))
ax.plot(min_dev_kendall_vertices, min_quantity_vertices, 'ro', label='Min BEGS')
ax.plot(0., quantity_indep, 'bo', label='Independence')
ax.legend(loc=0)
ax.set_xlabel('Kendall coefficient deviation')
ax.set_ylabel('Quantile at $\\alpha = %.2f$' % (alpha))
fig.tight_layout()
fig.savefig('./output/few-pairs/multidim_test_quantile_dim_%d_K_%d.pdf' % (dim, K))

In [ ]:
fig, axes = plt.subplots(dim-1, dim-1, figsize=(3*dim, 2.5*dim), sharex=True, sharey=True)

k = 0
for i in range(dim-1):
    for j in range(i+1):
        ax = axes[i, j] if dim > 2 else axes
        ax.plot(kendalls_lhs[:, k], quantities_lhs, 'g.', label='EGS K=%d' % (K))
        ax.plot(min_kendall_lhs[k], min_quantity_lhs, 'go', label='Min EGS')
        ax.plot(kendalls_vertices[:, k], quantities_vertices, 'r.', label='BEGS K=%d' % (min(K, 3**dim-1)))
        ax.plot(min_kendall_vertices[k], min_quantity_vertices, 'ro', label='Min BEGS')
        ax.plot(0., quantity_indep, 'bo', label='Independence')
        k += 1
        if i == dim-2:
            ax.set_xlabel('Kendall coefficient')
        if j == 0:
            ax.set_ylabel('Quantile at $\\alpha = %.2f$' % (alpha))
        if i == j+1:
            ax.legend(loc=0)
            
fig.tight_layout()
fig.savefig('./output/few-pairs/matrix_plot_multidim_test_quantile_dim_%d_K_%d.pdf' % (dim, K))

In [ ]:
plot_iterative_results(iterative_results, indep_result=indep_result, grid_results=grid_results_lhs, q_func=q_func)
plt.savefig('./output/few-pairs/iterative_result_with_grid_%d.pdf' % (K))